#**Smart Loan Recovery System**

-To build a loan recovery system with Machine Learning, we will use a dataset containing borrower profiles, loan details, and repayment histories.

**Importing the libraries and dataset**

In [ ]:
import pandas as pd
df = pd.read_csv('loan-recovery.csv')
df.head()

,Borrower_ID,Age,Gender,Employment_Type,Monthly_Income,Num_Dependents,Loan_ID,Loan_Amount,Loan_Tenure,Interest_Rate,...,Collateral_Value,Outstanding_Loan_Amount,Monthly_EMI,Payment_History,Num_Missed_Payments,Days_Past_Due,Recovery_Status,Collection_Attempts,Collection_Method,Legal_Action_Taken
0,BRW_1,59,Male,Salaried,215422,0,LN_1,1445796,60,12.39,...,1.727997e+06,2.914130e+05,4856.88,On-Time,0,0,Partially Recovered,1,Settlement Offer,No
1,BRW_2,49,Female,Salaried,60893,0,LN_2,1044620,12,13.47,...,1.180032e+06,6.652042e+05,55433.68,On-Time,0,0,Fully Recovered,2,Settlement Offer,No
2,BRW_3,35,Male,Salaried,116520,1,LN_3,1923410,72,7.74,...,2.622540e+06,1.031372e+06,14324.61,Delayed,2,124,Fully Recovered,2,Legal Notice,No
3,BRW_4,63,Female,Salaried,140818,2,LN_4,1811663,36,12.23,...,1.145493e+06,2.249739e+05,6249.28,On-Time,1,56,Fully Recovered,2,Calls,No
4,BRW_5,28,Male,Salaried,76272,1,LN_5,88578,48,16.13,...,0.000000e+00,3.918989e+04,816.46,On-Time,1,69,Fully Recovered,0,Debt Collectors,No


In [ ]:
df.describe()

,Age,Monthly_Income,Num_Dependents,Loan_Amount,Loan_Tenure,Interest_Rate,Collateral_Value,Outstanding_Loan_Amount,Monthly_EMI,Num_Missed_Payments,Days_Past_Due,Collection_Attempts
count,500.000000,500.000000,500.000000,5.000000e+02,500.00000,500.000000,5.000000e+02,5.000000e+02,500.000000,500.000000,500.000000,500.000000
mean,43.116000,134829.920000,1.476000,1.024907e+06,46.10400,11.192820,6.032240e+05,5.627260e+05,15861.536020,1.912000,70.678000,3.000000
std,12.733217,68969.356746,1.145447,5.907556e+05,18.23706,3.775209,7.457131e+05,4.723581e+05,18709.231315,2.110252,60.211038,2.807805
min,21.000000,15207.000000,0.000000,5.413800e+04,12.00000,5.020000,0.000000e+00,1.571283e+04,261.880000,0.000000,0.000000,0.000000
25%,32.000000,76343.250000,0.000000,4.629848e+05,36.00000,7.907500,0.000000e+00,1.822072e+05,4039.097500,1.000000,4.000000,1.000000
50%,44.000000,134929.500000,1.000000,9.971240e+05,48.00000,10.915000,2.327684e+05,4.133240e+05,9330.170000,2.000000,66.500000,2.000000
75%,53.000000,193086.250000,3.000000,1.557952e+06,60.00000,14.577500,1.111106e+06,8.324787e+05,20439.485000,3.000000,122.250000,4.000000
max,64.000000,249746.000000,3.000000,1.995325e+06,72.00000,17.970000,2.744395e+06,1.932396e+06,127849.230000,12.000000,180.000000,10.000000


Analyzing Data Distribution and Relationships


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

fig = px.histogram(df,x="Loan_Amount",nbins=30,marginal="violin", opacity=0.7,
                   title = "Loan Amount Distribution & Relationship with Monthly Income",
                   labels = {'Loan_Amount':'Loan Amount (in $)','Monthly_Income':"Monthly Income"},
                   color_discrete_sequence=["royalblue"])

fig.add_trace(go.Scatter(
    x=sorted(df['Loan_Amount']),
    y=px.histogram(df, x='Loan_Amount', nbins=30, histnorm='probability density').data[0]['y'],
    mode='lines',
    name='Density Curve',
    line=dict(color='red', width=2)
))

scatter = px.scatter(df, x='Loan_Amount', y='Monthly_Income',
                     color='Loan_Amount', color_continuous_scale='Viridis',
                     size=df['Loan_Amount'], hover_name=df.index)

for trace in scatter.data:
    fig.add_trace(trace)

fig.update_layout(
    annotations=[
        dict(
            x=max(df['Loan_Amount']) * 0.8, y=max(df['Monthly_Income']),
            text="Higher Loan Amounts are linked to Higher Income Levels",
            showarrow=True,
            arrowhead=2,
            font=dict(size=12, color="red"
            )
        )
    ],
    xaxis_title="Loan Amount (in $)",
    yaxis_title="Monthly Income (in $)",
    template="plotly_white",
    showlegend=True
)

fig.show()
%matplotlib inline



Analyzing Payment History


In [ ]:
df.Payment_History.value_counts()

,count
Payment_History,
On-Time,306
Delayed,140
Missed,54


In [ ]:
fig = px.bar(df,
                   x = "Payment_History",color="Recovery_Status",barmode="group",
                   title="How Payment History Affects Loan Recovery Status",
                   labels={"Payment_History": "Payment History", "count": "Number of Loans"},
                   color_discrete_map={"Recovered": "green", "Not Recovered": "red"})
fig.update_layout(
    xaxis_title="Payment History",
    yaxis_title="Number of Loans",
    legend_title="Recovery Status",
  template="plotly" )
fig.show()


In [ ]:
fig = px.box(df,
    x = "Recovery_Status",y="Num_Missed_Payments",color="Recovery_Status",
    title="How Missed PAyments Affect Loan Recovery Status",
    labels={"Num_Missed_Payments": "Number of Missed Payments", "Recovery_Status": "Recovery Status"},
    color_discrete_map={"Recovered": "green", "Not Recovered": "red"}
    )
fig.update_layout(
    xaxis_title="Recovery Status",
    yaxis_title="Number of Missed Payments",
    template="ggplot2"
)
fig.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

# Standardize the data
features = ['Age', 'Monthly_Income', 'Loan_Amount', 'Loan_Tenure', 'Interest_Rate',
            'Collateral_Value', 'Outstanding_Loan_Amount', 'Monthly_EMI', 'Num_Missed_Payments', 'Days_Past_Due']

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[features])




In [ ]:
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['Borrower_Segment'] = kmeans.fit_predict(df_scaled)

fig = px.scatter(df, x='Monthly_Income', y='Loan_Amount',
                 color=df['Borrower_Segment'].astype(str), size='Loan_Amount',
                 hover_data={'Monthly_Income': True, 'Loan_Amount': True, 'Borrower_Segment': True},
                 title="Borrower Segments Based on Monthly Income and Loan Amount",
                 labels={"Monthly_Income": "Monthly Income ($)", "Loan_Amount": "Loan Amount ($)", "Borrower_Segment": "Segment"},
                 color_discrete_sequence=px.colors.qualitative.Vivid)

fig.add_annotation(
    x=df['Monthly_Income'].mean(), y=df['Loan_Amount'].max(),
    text="Higher loans are clustered in specific income groups",
    showarrow=True,
    arrowhead=2,
    font=dict(size=12, color="red")
)

fig.update_layout(
    xaxis_title="Monthly Income ($)",
    yaxis_title="Loan Amount ($)",
    template="seaborn",
    legend_title="Borrower Segment"
)

fig.show()
df

,Borrower_ID,Age,Gender,Employment_Type,Monthly_Income,Num_Dependents,Loan_ID,Loan_Amount,Loan_Tenure,Interest_Rate,...,Outstanding_Loan_Amount,Monthly_EMI,Payment_History,Num_Missed_Payments,Days_Past_Due,Recovery_Status,Collection_Attempts,Collection_Method,Legal_Action_Taken,Borrower_Segment
0,BRW_1,59,Male,Salaried,215422,0,LN_1,1445796,60,12.39,...,2.914130e+05,4856.88,On-Time,0,0,Partially Recovered,1,Settlement Offer,No,3
1,BRW_2,49,Female,Salaried,60893,0,LN_2,1044620,12,13.47,...,6.652042e+05,55433.68,On-Time,0,0,Fully Recovered,2,Settlement Offer,No,1
2,BRW_3,35,Male,Salaried,116520,1,LN_3,1923410,72,7.74,...,1.031372e+06,14324.61,Delayed,2,124,Fully Recovered,2,Legal Notice,No,3
3,BRW_4,63,Female,Salaried,140818,2,LN_4,1811663,36,12.23,...,2.249739e+05,6249.28,On-Time,1,56,Fully Recovered,2,Calls,No,3
4,BRW_5,28,Male,Salaried,76272,1,LN_5,88578,48,16.13,...,3.918989e+04,816.46,On-Time,1,69,Fully Recovered,0,Debt Collectors,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,BRW_496,46,Female,Salaried,248483,1,LN_496,740796,72,16.59,...,4.135285e+05,5743.45,Delayed,2,169,Fully Recovered,2,Settlement Offer,No,2
496,BRW_497,30,Female,Salaried,243590,3,LN_497,1408126,60,11.03,...,3.173740e+05,5289.57,Delayed,2,102,Partially Recovered,9,Settlement Offer,No,2
497,BRW_498,46,Female,Salaried,113864,2,LN_498,375203,48,9.16,...,3.300302e+05,6875.63,Delayed,3,140,Fully Recovered,3,Settlement Offer,No,2
498,BRW_499,54,Male,Salaried,158401,2,LN_499,1769890,24,11.19,...,1.565339e+06,65222.46,On-Time,3,9,Partially Recovered,9,Calls,No,1


In [ ]:
# updating segment names
df['Segment_Name'] = df['Borrower_Segment'].map({
    0: 'Moderate Income, High Loan Burden',
    1: 'High Income, Low Default Risk',
    2: 'Moderate Income, Medium Risk',
    3: 'High Loan, Higher Default Risk'
})
df

,Borrower_ID,Age,Gender,Employment_Type,Monthly_Income,Num_Dependents,Loan_ID,Loan_Amount,Loan_Tenure,Interest_Rate,...,Monthly_EMI,Payment_History,Num_Missed_Payments,Days_Past_Due,Recovery_Status,Collection_Attempts,Collection_Method,Legal_Action_Taken,Borrower_Segment,Segment_Name
0,BRW_1,59,Male,Salaried,215422,0,LN_1,1445796,60,12.39,...,4856.88,On-Time,0,0,Partially Recovered,1,Settlement Offer,No,3,"High Loan, Higher Default Risk"
1,BRW_2,49,Female,Salaried,60893,0,LN_2,1044620,12,13.47,...,55433.68,On-Time,0,0,Fully Recovered,2,Settlement Offer,No,1,"High Income, Low Default Risk"
2,BRW_3,35,Male,Salaried,116520,1,LN_3,1923410,72,7.74,...,14324.61,Delayed,2,124,Fully Recovered,2,Legal Notice,No,3,"High Loan, Higher Default Risk"
3,BRW_4,63,Female,Salaried,140818,2,LN_4,1811663,36,12.23,...,6249.28,On-Time,1,56,Fully Recovered,2,Calls,No,3,"High Loan, Higher Default Risk"
4,BRW_5,28,Male,Salaried,76272,1,LN_5,88578,48,16.13,...,816.46,On-Time,1,69,Fully Recovered,0,Debt Collectors,No,0,"Moderate Income, High Loan Burden"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,BRW_496,46,Female,Salaried,248483,1,LN_496,740796,72,16.59,...,5743.45,Delayed,2,169,Fully Recovered,2,Settlement Offer,No,2,"Moderate Income, Medium Risk"
496,BRW_497,30,Female,Salaried,243590,3,LN_497,1408126,60,11.03,...,5289.57,Delayed,2,102,Partially Recovered,9,Settlement Offer,No,2,"Moderate Income, Medium Risk"
497,BRW_498,46,Female,Salaried,113864,2,LN_498,375203,48,9.16,...,6875.63,Delayed,3,140,Fully Recovered,3,Settlement Offer,No,2,"Moderate Income, Medium Risk"
498,BRW_499,54,Male,Salaried,158401,2,LN_499,1769890,24,11.19,...,65222.46,On-Time,3,9,Partially Recovered,9,Calls,No,1,"High Income, Low Default Risk"


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df['High_Risk_Flag'] = df['Segment_Name'].apply(lambda x: 1 if x in ['High Loan, Higher Default Risk', 'Moderate Income, High Loan Burden'] else 0)

# selecting features for the model
features = ['Age', 'Monthly_Income', 'Loan_Amount', 'Loan_Tenure', 'Interest_Rate',
            'Collateral_Value', 'Outstanding_Loan_Amount', 'Monthly_EMI', 'Num_Missed_Payments', 'Days_Past_Due']
X = df[features]
y = df['High_Risk_Flag']

# splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# training the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# get risk scores (probability of being high risk)
risk_scores = rf_model.predict_proba(X_test)[:, 1]  # probability of high risk

# add risk scores and flags to test data
df_test = X_test.copy()
df_test['Risk_Score'] = risk_scores
df_test['Predicted_High_Risk'] = (df_test['Risk_Score'] > 0.5).astype(int)  # Threshold at 50% probability

# merging with borrower details
df_test = df_test.merge(df[['Borrower_ID', 'Segment_Name', 'Recovery_Status', 'Collection_Method', 'Collection_Attempts', 'Legal_Action_Taken']],
                        left_index=True, right_index=True)

In [ ]:
# creating a new column for the dynamic recovery strategy based on risk scores
def assign_recovery_strategy(risk_score):
    if risk_score > 0.75:
        return "Immediate legal notices & aggressive recovery attempts"
    elif 0.50 <= risk_score <= 0.75:
        return "Settlement offers & repayment plans"
    else:
        return "Automated reminders & monitoring"

df_test['Recovery_Strategy'] = df_test['Risk_Score'].apply(assign_recovery_strategy)

df_test.head()

,Age,Monthly_Income,Loan_Amount,Loan_Tenure,Interest_Rate,Collateral_Value,Outstanding_Loan_Amount,Monthly_EMI,Num_Missed_Payments,Days_Past_Due,Risk_Score,Predicted_High_Risk,Borrower_ID,Segment_Name,Recovery_Status,Collection_Method,Collection_Attempts,Legal_Action_Taken,Recovery_Strategy
436,25,156826,1510049,60,16.22,8.373402e+05,3.296001e+05,5493.34,1,179,0.42,0,BRW_437,"High Loan, Higher Default Risk",Partially Recovered,Legal Notice,7,No,Automated reminders & monitoring
448,57,33752,1803767,12,8.30,0.000000e+00,4.411477e+05,36762.31,0,0,0.64,1,BRW_449,"Moderate Income, High Loan Burden",Fully Recovered,Calls,2,No,Settlement offers & repayment plans
88,27,71503,1898058,24,12.59,1.087594e+06,4.227607e+05,17615.03,2,179,0.76,1,BRW_89,"High Loan, Higher Default Risk",Fully Recovered,Debt Collectors,3,No,Immediate legal notices & aggressive recovery ...
145,59,215641,1492181,36,15.85,1.340937e+06,1.337482e+06,37152.29,3,125,0.32,0,BRW_146,"High Income, Low Default Risk",Fully Recovered,Calls,3,No,Automated reminders & monitoring
344,52,106658,1958911,36,12.09,2.128380e+06,1.592191e+06,44227.52,0,0,0.09,0,BRW_345,"High Income, Low Default Risk",Fully Recovered,Calls,0,No,Automated reminders & monitoring


from matplotlib import pyplot as plt
_df_0['Age'].plot(kind='hist', bins=20, title='Age')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Monthly_Income'].plot(kind='hist', bins=20, title='Monthly_Income')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Loan_Amount'].plot(kind='hist', bins=20, title='Loan_Amount')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Loan_Tenure'].plot(kind='hist', bins=20, title='Loan_Tenure')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['Segment_Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='Age', y='Segment_Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_22['Recovery_Status'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_22, x='Age', y='Recovery_Status', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_23['Collection_Method'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_23, x='Age', y='Collection_Method', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)